In [1]:
import pandas as pd
import json
# import numpy as np
from os import listdir
from ete3 import Tree


In [2]:
#### get list of files for relax folder ####
path="/home/maeperez/scratch/Clams/hyphy2.5/absrel"
genes=[]
for file in listdir(path):
    if file[-5:]==".json":
        genes+=[str(file.split('.')[0])]

genes=sorted(genes)

pval_threshold=0.05

#### init all dictionaries and functions ####
branches_dic={'Bathy_MS2': 'Bathy',
 'SUP05_SP6': 'SUP05',
 'R_fausta_': 'R.fausta',
 'R_magnifi': 'R.magnifica',
 'R_pacific': 'R.pacifica',
 'R_phaseol': 'R.phaseoliformis',
 'R_pliocar': 'R.pliocardia',
 'R_rectima': 'R.rectimargo',
 'R_southwa': 'R.southwardae',
 'V_diagona': 'V.diagonalis',
 'V_extenta': 'V.extenta',
 'V_gigas1_': 'V.gigas1',
 'V_gigas2_': 'V.gigas2',
 'V_marissi': 'V.marissinica',
 'V_okutani': 'V.okutanii',
 'V_soyoae1': 'V.soyoae1',
 'V_soyoae2': 'V.soyoae2'}

fl=['Bathy_MS2', 'SUP05_SP6']
ruthia=['R_fausta_', 'R_magnifi', 'R_pacific', 'R_phaseol', 'R_pliocar', 'R_rectima', 'R_southwa']
gigas=['V_diagona', 'V_extenta', 'V_gigas1_', 'V_gigas2_', 'V_marissi', 'V_okutani', 'V_soyoae1', 'V_soyoae2']
partitions={'a':[set(fl),set(branches_dic.keys())-set(fl)],
    'b':[set(ruthia),set(branches_dic.keys())-set(ruthia)],
    'c':[set(gigas),set(branches_dic.keys())-set(gigas)],
    'd':[set(['V_okutani', 'V_soyoae1', 'V_soyoae2']),set(branches_dic.keys())-set(['V_okutani', 'V_soyoae1', 'V_soyoae2'])],
    'e':[set(['V_diagona', 'V_extenta', 'V_gigas1_', 'V_gigas2_']),set(branches_dic.keys())-set(['V_diagona', 'V_extenta', 'V_gigas1_', 'V_gigas2_'])],
    'f':[set(['V_diagona', 'V_extenta']),set(branches_dic.keys())-set(['V_diagona', 'V_extenta'])],
    'g':[set(['R_fausta_', 'R_pacific', 'R_phaseol', 'R_pliocar', 'R_rectima', 'R_southwa']),set(branches_dic.keys())-set(['R_fausta_', 'R_pacific', 'R_phaseol', 'R_pliocar', 'R_rectima', 'R_southwa'])],
    'h':[set(['R_fausta_', 'R_pacific', 'R_phaseol', 'R_rectima']),set(branches_dic.keys())-set(['R_fausta_', 'R_pacific', 'R_phaseol', 'R_rectima'])],
    'i':[set(['R_fausta_', 'R_pacific', 'R_rectima']),set(branches_dic.keys())-set(['R_fausta_', 'R_pacific', 'R_rectima'])],
    'j':[set(['R_pliocar', 'R_southwa']),set(branches_dic.keys())-set(['R_pliocar', 'R_southwa'])],
    'k':[set(['V_soyoae1', 'V_soyoae2']),set(branches_dic.keys())-set(['V_soyoae1', 'V_soyoae2'])],
    'l':[set(['V_gigas1_', 'V_gigas2_']),set(branches_dic.keys())-set(['V_gigas1_', 'V_gigas2_'])],
    'm':[set(['V_diagona', 'V_extenta', 'V_gigas1_', 'V_gigas2_', 'V_okutani', 'V_soyoae1', 'V_soyoae2']),set(branches_dic.keys())-set(['V_diagona', 'V_extenta', 'V_gigas1_', 'V_gigas2_', 'V_okutani', 'V_soyoae1', 'V_soyoae2'])],
    'n':[set(['R_fausta_', 'R_rectima']),set(branches_dic.keys())-set(['R_fausta_', 'R_rectima'])]}

def find_node_partition(br,tree_str):
    t = Tree(tree_str+';', format=1) # load ete3 trees from tree string
    node=t&br # go to node in interest
    leaves=set([l[:9] for l in node.get_leaf_names()]) # get all leaves for that node
    category='NA'
    for k,v in partitions.items(): # find category associated to that leaves set
        if leaves in v:
            category=k
            break
    return(category)

df=pd.DataFrame([],columns=['gene_id']+list(partitions.keys())+list(branches_dic.values()))

#####################

branches=[]
for gene in genes:
#     print(gene)
    remote_file = open(path+'/'+gene+'.absrel.json')
    absrel_json = json.load(remote_file)
#     print(absrel_json.keys())
    if pval_threshold==0.05:
        if absrel_json['test results']["positive test results"]==0:
            continue
    new_row={'gene_id':gene}
    for branch in absrel_json['branch attributes']['0'].keys():
        branches+=[branch]
        branch_pval=absrel_json['branch attributes']['0'][branch]['Corrected P-value']
        if branch_pval<pval_threshold:
            br=branch[:9]
            if br[:4]=='Node':
                tree_str=absrel_json['input']['trees']['0']
                cat=find_node_partition(br,tree_str)
                new_row[cat]=1

            else:
                cat=branches_dic[br]
                new_row[cat]=1
    if len(new_row.keys())>1:
        df=df.append(new_row,ignore_index=True)


In [7]:
df[~df['NA'].isna()]

,gene_id,a,b,c,d,e,f,g,h,i,...,R.southwardae,V.diagonalis,V.extenta,V.gigas1,V.gigas2,V.marissinica,V.okutanii,V.soyoae1,V.soyoae2,NA
2,Rmag_0080,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12,Rmag_0197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
24,Rmag_0325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
53,Rmag_0545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
61,Rmag_0611,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
66,Rmag_0721,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
76,Rmag_0834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
78,Rmag_0837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
79,Rmag_0838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
90,Rmag_0929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [6]:
print(len(df))

104


In [ ]:
d=read_csv(pat)